# nomaly detection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
df = pd.read_csv("hearing_test.csv")
df

,age,physical_score,test_result
0,33.0,40.7,1
1,50.0,37.2,1
2,52.0,24.7,0
3,56.0,31.0,0
4,35.0,42.9,1
...,...,...,...
4995,73.0,3.9,0
4996,57.0,33.9,1
4997,49.0,34.5,1
4998,38.0,46.4,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             5000 non-null   float64
 1   physical_score  5000 non-null   float64
 2   test_result     5000 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 117.3 KB


In [4]:
df.describe()

,age,physical_score,test_result
count,5000.000000,5000.000000,5000.000000
mean,51.609000,32.760260,0.600000
std,11.287001,8.169802,0.489947
min,18.000000,-0.000000,0.000000
25%,43.000000,26.700000,0.000000
50%,51.000000,35.300000,1.000000
75%,60.000000,38.900000,1.000000
max,90.000000,50.000000,1.000000


In [5]:
df['test_result'].value_counts()

1    3000
0    2000
Name: test_result, dtype: int64

# Data Processing

In [17]:
df_passed = df[df['test_result'] == 1]
df_failed = df[df['test_result'] == 0]

# collect only 2000 records randomly
df_under = df_under.sample(2000)

# build a new dataset using 2000 passed and 2000 faild records
df_balanced = pd.concat([df_under, df_failed], axis = 0)

In [18]:
df_balanced['test_result'].value_counts()

1    2000
0    2000
Name: test_result, dtype: int64

In [20]:
x = df_balanced.drop('test_result', axis = 1)
y = df_balanced['test_result']


In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 12345)

#### model building

In [22]:
from sklearn.svm import SVC
model = SVC()
model.fit(x_train, y_train)

SVC()

In [34]:
y_true = y_test
y_pred = model.predict(x_test)

In [35]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true, y_pred)
accuracy

0.9183333333333333

In [25]:
## we did not do balancing beacause here the output is same for both balancing un-balance model (accuracy)

### fine tuning the model

In [41]:
from sklearn.model_selection import GridSearchCV
parameters = {
    "kernel" : ('kernel', 'rbf', 'linear', 'poly'),
     "C": range(1, 10),
     "gamma": ('scale', 'auto')
}
gs = GridSearchCV(model, parameters)

gs.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

GridSearchCV(estimator=SVC(),
             param_grid={'C': range(1, 10), 'gamma': ('scale', 'auto'),
                         'kernel': ('kernel', 'rbf', 'linear', 'poly')})

In [44]:
gs.cv_results_

{'mean_fit_time': array([3.17573547e-04, 1.92942619e-02, 5.59985638e-02, 7.07121849e-02,
        2.13956833e-04, 8.71441841e-02, 5.47475338e-02, 1.90403413e+01,
        2.23636627e-04, 1.90442562e-02, 8.47303867e-02, 1.11282778e-01,
        2.20394135e-04, 8.84992599e-02, 8.45746040e-02, 2.57596222e+01,
        2.33364105e-04, 1.93681717e-02, 1.14519644e-01, 1.46362114e-01,
        2.20108032e-04, 8.76426697e-02, 1.14903927e-01, 2.74641578e+01,
        2.33554840e-04, 2.00610161e-02, 1.34826660e-01, 1.70734882e-01,
        2.26306915e-04, 8.73392582e-02, 1.35096884e-01, 2.50011278e+01,
        2.34794617e-04, 1.86714649e-02, 1.55121374e-01, 1.96433830e-01,
        2.23445892e-04, 8.53183746e-02, 1.55112457e-01, 2.15729435e+01,
        2.43425369e-04, 1.99251652e-02, 1.81807613e-01, 2.32259178e-01,
        2.32505798e-04, 8.94433975e-02, 1.81406832e-01, 2.13783767e+01,
        2.36940384e-04, 1.98071003e-02, 1.96859980e-01, 2.65623569e-01,
        2.36415863e-04, 8.93848896e-02, 1.97025

# cross validation

In [32]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, x_train, y_train, cv = 6)
scores

array([0.90578158, 0.91220557, 0.9143469 , 0.91006424, 0.91845494,
       0.92274678])

In [36]:
# accuracy and cross validation are should same or near to each other it indicate validation is currect

In [45]:
# to fine tune the model and to find optimal values
# gridsearchcv do the permutation combination of data it is more accurate
# random searchcv also do permutation combination but with some random values it pick from data and do the operation 
